In [17]:
import spacy
import pandas as pd
import swifter
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

nlp = spacy.load("en_core_web_trf")

In [19]:

def preprocess_name(name):
    name = re.sub(r"(\b\w+)\.(?=\w)", r"\1. ", name)  # Add space after honorifics
    return name

def clean_name(name):
    if not isinstance(name, str) or not name.strip():
        return "", ""

    name = preprocess_name(name)
    doc = nlp(name)

    name_parts = [token.text for token in doc if token.ent_type_ == "PERSON" or token.pos_ == "PROPN"]

    # Remove honorifics if they appear as separate words
    name_parts = [part for part in name_parts if part.lower() not in HONORIFICS_WITH_DOTS]

    first_name = re.sub(r'^[^A-Za-z]+|[^A-Za-z]+$', '', name_parts[0]).capitalize() if name_parts else ""
    last_name = re.sub(r'^[^A-Za-z]+|[^A-Za-z]+$', '', name_parts[-1]).capitalize() if len(name_parts) > 1 else ""

    return first_name, last_name

def process_csv(input_file, output_file):
    df = pd.read_csv(input_file, low_memory=False)

    df["First Name"], df["Last Name"] = zip(*df[name_column].swifter.apply(clean_name))

    df.to_csv(output_file, index=False)

In [ ]:

HONORIFICS = {
    "mr", "mrs", "ms", "miss", "mx", "dr", "prof", "sir", "madam", "lord", "lady", "dame", "shri", "sri", "smt", "jr", "sr", "ii", "iii", "iv", "v", "col", "colonel", "lt", "ltc", "ltcol", "maj", 
    "cpt", "capt", "cmdr", "sgt", "sgtmj", "sgtmaj", "sergeant", "pfc", "cpl", "wo", "brig", "gen", "adm", "fso", "rearadm", "viceadm", "chief", "cmd", "comdt", "aircm", "airm", "com", "bgen", "chf", 
    "king", "queen", "prince", "princess", "duchess", "baroness", "earl", "viscount", "count", "countess", "marquis", "marquess", "emperor", "empress", "pres", "gov", "sen", "rep", "mp", "mla", 
    "mlc", "mep", "mayor", "amb", "consul", "md", "do", "dds", "dmd", "od", "dpt", "dnp", "psyd", "pharmd", "mbbs", "bhms", "bams", "bds", "dm", "chb", "frcp", "frcs", "faafp", "facep", "facs", "fasge", 
    "facc", "faan", "faanp", "frcog", "fcohns", "fanzca", "fca", "fccp", "fccm", "fccs", "fcnp", "fchc", "fcps", "frca", "jd", "esq", "llb", "llm", "lld", "bcl", "dcl", "qc", "kc", "qcs", "mba", "bba", 
    "cpa", "cfa", "cfp", "cma", "chfc", "cia", "cisa", "cfe", "crm", "crma", "prm", "cmt", "cfr", "cbv", "cea", "crisc", "cgeit", "frsa", "frm", "cmc", "cmgr", "caia", "asa", "pe", "ce", "me", "ee", "se", 
    "cswp", "cissp", "cism", "pmp", "itil", "mcsa", "mcsd", "mct", "aws", "gcp", "azure", "oca", "ocp", "redhat", "cspo", "sa", "safe", "cca", "ccna", "ccnp", "ccie", "scjp", "ocpjp", "scp", "security", 
    "network", "linux", "pentest", "casp", "ceh", "chfi", "oscp", "gsec", "gpen", "csm", "scrum", "ccsk", "phd", "edd", "msc", "ma", "ba", "bs", "bsc", "bfa", "mfa", "mlis", "med", "msw", "mph", "mpp", 
    "mpa", "mdiv", "thm", "mtheol", "msed", "mres", "mpaed", "mteach", "dsc", "dlitt", "dlaw", "dtech", "mphil", "bphil", "dphil", "scd", "rn", "lpn", "rph", "rpa", "cna", "np", "cnm", "crna", "dph", 
    "ot", "mot", "rd", "ld", "cns", "cph", "fnp", "agnp", "anp", "gnp", "pmhnp", "whnp", "acnp", "ccht", "chs", "cht", "ccht-a", "crrn", "cnn", "ccrn", "chpn", "ccn", "ctn", "cptc", "csts", "cfle", "cctc", 
    "cpnp", "nnp", "acrn", "ahn", "ccp", "ccs", "ccs-p", "ccma", "rsc", "cscs", "chm", "cgs", "pgeo", "gisp", "aqe", "cwem", "ciwem", "cenv", "mcibse", "michem", "mice", "miema", "mcsi", "mciht", "miob", 
    "mrics", "mrtpi", "mciwm", "frsc", "fo", "atp", "cfii", "ame", "a&p", "dp", "ssp", "dpa", "dmr", "mni", "chmm", "cmm", "dmm", "cms", "chop", "chc", "chsp", "cipm", "fapm", "ceo", "deacon", "dcn", 
    "pastor", "reverend", "rev", "minister", "min", "captain", "evangelist", "preacher", "apostle", "dj", "mc", "vfx", "gfx", "sfx", "fx", "dga", "sag", "aftra", "ascap", "bmi", "sesac", "nsc", 
    "oscar", "grammy", "emmy", "chfm", "chp", "fache", "fahra", "fashra", "cchem", "cm", "cpp", "cppa", "cspa", "cfm", "cfmi", "crsp", "csr", "fsp", "fsd", "hsd", "rdh", "rpc", "dbe", "obe", "mbe", "gbe", 
    "kbe", "cbe", "ch", "pcs", "ias", "ifs", "ips", "irs", "mpse", "pcc", "cppc", "crnp", "admin", "architect", "aspnccer", "associate", "cdfatm", "cdfom", "cdpse", "ciampi", "cic", "cicp", "cid", "cimic", 
    "cippus", "cippuse", "clssgb", "cmtse", "cossasp", "cpccts", "crcst", "crisci", "crisp", "cssbb", "cssgb", "cts-d", "cts-i", "cvpcc", "devlp", "fnp-c", "mba-cpa", "pa-c", "pmi-acp", "psw-c", "shrm-cp", 
    "vp-cissp", "mbahrm", "mba-hr", "aic-m", "mba-hc", "mbaha", "mbamha", "mdmba", "mbec", "mboe", "mcse", "mba-hm", "mcj", "mbt", "mabc", "psm", "psp", "popm", "pls", "pcip", "pfmp", "phr", "prc", "psc", 
    "pmc", "pcam", "pbc", "picq", "pga", "pgmp", "pws", "phn", "ricp", "fsgm", "rbsm", "rost", "rex", "rhit", "rdn", "qssp", "rcdd", "isr", "fma", "ohst", "rehs", "iws", "inc", "jnr", "gsp", "rvt", "rhia", 
    "edfp", "recla", "otr-l", "erp", "fsa", "facp", "fscp", "olp", "fmp", "iqc", "emt-p", "itilv", "rrp", "rix", "osp", "rde", "rrt", "frs", "otrl", "icm", "rwcs", "rtm", "rohr", "enz", "fpc", "rse", "rohm", 
    "itpm", "mahrd", "fcsi", "mn-e", "erb", "qpa", "gphr", "euw", "ess", "imh-e", "qpfc", "rdcs", "gms", "rfc", "oms", "rn-bc", "regt", "fmo", "fhm", "grp", "grcp", "rvs", "rohn", "rem", "ilg", "jmac", "fpa", 
    "chhr", "shrm-spc", "lcadc", "rn-bsn", "bcba", "msn rn - bc", "ms-mgmt", "cxa", "lcsw", "jd/mba", "csbb", "hrm", "cnor", "cprw", "pharm", "cipd", "cgfm", "cde", "ccue", "clss-hc", "lcsw-r", "clcp", 
    "rnbsccm", "cota/l", "bcgp", "rnc-nic", "rnc-ob", "president", "instructor", "remember", "recruiter", "advisor", "director", "relations", "trading", "manager", "program", "operation", "association", 
    "hospital", "sphr", "accounting", "shrm", "veteran", "executive", "ssm", "experience", "constant", "fitness", "center", "centre", "summit", "expert", "market", "health", "spsc", "phrca", 
    "certified", "consultant", "effective", "medical", "device", "supervisor", "cism cisa cissp", "lssbb", "acca", "cam", "cos", "mcc", "mcr", "cit", "osb", "arm", "ing", "rid", "rcs", "rls", "pta", "ply", 
    "spc", "sgm", "asla", "nha", "cep", "clu", "flmi", "cet", "ala", "der", "ahr", "cih", "cima", "cir", "mcp", "mcs", "mbm", "mbcp", "mcsm", "dvs", "dvn", "dvm", "dtm", "dsa", "drph", "dpm", "dmcp", "dma", 
    "dhsc", "dha", "dbia", "dba", "cyr", "cws", "cwi", "cwe", "cwd", "cwcm", "cui", "ctsm", "cts", "ctrs", "ctr", "ctp", "ctie", "ctfa", "cte", "ctb", "cta", "csw", "cst", "css", "cspm", "csp", "csme", 
    "csl", "csi", "cshm", "csee", "cscp", "crx", "crs", "crps", "crpc", "crp", "crme", "crl", "crfp", "crcr", "crcm", "crc", "crb", "cqe", "cqa", "cpsm", "cps", "cprs", "cprp", "cprm", "cppo", "cppm", 
    "cpo", "cpmr", "cpmm", "cpma", "cpm", "cpim", "cpia", "cpht", "cpfs", "cpfo", "cpe", "cpdw", "cpcu", "cpc", "cnp", "cnmt", "cmte", "cmsp", "cmrp", "cmps", "cmp", "cmlt", "cmip", "cmhs", "cmfc", "cme", 
    "cmca", "cmb", "cltd", "cltc", "clpm", "clp", "clm", "clcs", "citp", "cisr", "ciso", "cis", "chtm", "chst", "chrm", "chpa", "chg", "chfp", "chdm", "chcp", "cgsp", "cgcs", "cgc", "cfsp", "cfs", "cfps", 
    "cfi", "cfer", "cfd", "cfcm", "ces", "cem", "cec", "cebs", "cdt", "cds", "cdr", "cdpe", "cdp", "cdmp", "cdm", "cdip", "cdia", "cdfa", "cdbc", "ccxp", "ccwp", "ccte", "ccsp", "ccsm", "ccsl", "ccrp", 
    "ccrm", "ccra", "ccpm", "ccoa", "ccm", "ccim", "cchp", "ccda", "cbp", "cbet", "cbcp", "cbap", "cba", "caps", "capp", "capm", "capb", "capa", "cap", "cao", "canl", "cams", "cafs", "cadc", "awet", "awe", 
    "aud", "auci", "auch", "atc", "asp", "asis", "asid", "ashm", "ash", "ascp", "arx", "ark", "ari", "arb", "aprn", "apn", "aphr", "aos", "anh", "amo", "amft", "alst", "alc", "alba", "ais", "airc", "ains", 
    "aim", "aifd", "aifa", "aif", "aicp", "aic", "aia", "aft", "adr", "acsw", "acs", "acra", "acma", "ace", "aca", "abv", "abt", "absi", "aas", "eit", "pahm"
}

HONORIFICS_WITH_DOTS = HONORIFICS | {h + "." for h in HONORIFICS}

In [ ]:

name_column = "<ENTER COLUMN NAME>"

input_csv = "<ENTER FILE NAME>"
output_csv = "output.csv"

process_csv(input_csv, output_csv)